In [1]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import cantera as ct
import os
import gym 
import numpy as np 
from stable_baselines.common.policies import MlpPolicy, CnnLstmPolicy,MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common import set_global_seeds
from stable_baselines import ACKTR, PPO2
from envs.sim_env import SimEnv
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


```python
 def _train_step(self, learning_rate, cliprange, obs, returns, masks, actions, values, neglogpacs, update,
                    writer, states=None, cliprange_vf=None):
        """
        Training of PPO2 Algorithm
        :param learning_rate: (float) learning rate
        :param cliprange: (float) Clipping factor
        :param obs: (np.ndarray) The current observation of the environment
        :param returns: (np.ndarray) the rewards
        :param masks: (np.ndarray) The last masks for done episodes (used in recurent policies)
        :param actions: (np.ndarray) the actions
        :param values: (np.ndarray) the values
        :param neglogpacs: (np.ndarray) Negative Log-likelihood probability of Actions
        :param update: (int) the current step iteration
        :param writer: (TensorFlow Summary.writer) the writer for tensorboard
        :param states: (np.ndarray) For recurrent policies, the internal state of the recurrent model
        :return: policy gradient loss, value function loss, policy entropy,
                approximation of kl divergence, updated clipping range, training update operation
        :param cliprange_vf: (float) Clipping factor for the value function
        """
        advs = returns - values
        advs = (advs - advs.mean()) / (advs.std() + 1e-8)
        td_map = {self.train_model.obs_ph: obs, self.action_ph: actions,
                  self.advs_ph: advs, self.rewards_ph: returns,
                  self.learning_rate_ph: learning_rate, self.clip_range_ph: cliprange,
                  self.old_neglog_pac_ph: neglogpacs, self.old_vpred_ph: values}
        if states is not None:
            td_map[self.train_model.states_ph] = states
            td_map[self.train_model.dones_ph] = masks

        if cliprange_vf is not None and cliprange_vf >= 0:
            td_map[self.clip_range_vf_ph] = cliprange_vf

        if states is None:
            update_fac = self.n_batch // self.nminibatches // self.noptepochs + 1
        else:
            update_fac = self.n_batch // self.nminibatches // self.noptepochs // self.n_steps + 1

        if writer is not None:
            # run loss backprop with summary, but once every 10 runs save the metadata (memory, compute time, ...)
            if self.full_tensorboard_log and (1 + update) % 10 == 0:
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train],
                    td_map, options=run_options, run_metadata=run_metadata)
                writer.add_run_metadata(run_metadata, 'step%d' % (update * update_fac))
            else:
                summary, policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                    [self.summary, self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train],
                    td_map)
            writer.add_summary(summary, (update * update_fac))
        else:
            policy_loss, value_loss, policy_entropy, approxkl, clipfrac, _ = self.sess.run(
                [self.pg_loss, self.vf_loss, self.entropy, self.approxkl, self.clipfrac, self._train], td_map)

        return policy_loss, value_loss, policy_entropy, approxkl, clipfrac
```

In [4]:
def make_env(rank, log_dir, seed=0):
    """
    Utility function for multiprocessed env.
    
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
#         env = Monitor(SimEnv(), log_dir)
        env = SimEnv()
        env.seed(seed + rank)
        return env
    set_global_seeds(seed)
    return _init
num_cpu = 2
env = SubprocVecEnv([make_env(i, "") for i in range(num_cpu)])
model = PPO2(MlpLstmPolicy, env, gamma=0.99, n_steps=128, ent_coef=0.01, learning_rate=2.5e-4, vf_coef=0.5,
                 max_grad_norm=0.5, lam=0.95, nminibatches=2, noptepochs=4, cliprange=0.2, cliprange_vf=None,
                 verbose=0, tensorboard_log=None, _init_setup_model=True, policy_kwargs=None,
                 full_tensorboard_log=False)
model.learn(total_timesteps=50000)






Instructions for updating:
Use keras.layers.flatten instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [13]:
from stable_baselines.common.vec_env import DummyVecEnv

env = SubprocVecEnv([SimEnv, SimEnv])
obs = env.reset()
game_over = False
while not game_over: 
    action, _states = model.predict(obs)
    obs, rewards, game_over, info = env.step(action)
    env.render(mode='human')

ValueError: not enough values to unpack (expected 4, got 1)

In [12]:
%debug

> /home/edwin/anaconda3/envs/tf1_env/lib/python3.7/site-packages/stable_baselines/common/tile_images.py(15)tile_images()
     13     """
     14     img_nhwc = np.asarray(img_nhwc)
---> 15     n_images, height, width, n_channels = img_nhwc.shape
     16     # new_height was named H before
     17     new_height = int(np.ceil(np.sqrt(n_images)))

ipdb> img_nhwc
array([None], dtype=object)
ipdb> u
> /home/edwin/anaconda3/envs/tf1_env/lib/python3.7/site-packages/stable_baselines/common/vec_env/subproc_vec_env.py(134)render()
    132         imgs = [pipe.recv() for pipe in self.remotes]
    133         # Create a big image by tiling images from subprocesses
--> 134         bigimg = tile_images(imgs)
    135         if mode == 'human':
    136             import cv2

ipdb> self
ipdb> self.remotes
(<multiprocessing.connection.Connection object at 0x7f734262da50>,)
ipdb> q
